# Майнор "Прикладные задачи анализа данных"
## Домашнее задание 2 [10 баллов] до 23:59 22.03.2018. Предсказание цены акции по экономическим новостям


В этом домашнем задании вы попытаетесь предсказать рост цены акции компании Газпром по новостям о компании. Домашнее задание состоит из трех частей:
1. Предварительная обработка текстов и эксплоративный анализ
2. Baseline алгоритм
3. Творческая часть

Все три части можно считать независимыми – вы можете сделать одну или две из них, однако мы настоятельно советуем выполнить все три. Все инструкции по выполнению домашнего задания – ниже. 



Входные данные:
* Новости о компании "Газпром", начиная с 2010 года
* Стоимость акций компании "Газпром" на ММВБ, начиная с 2010 года
    * цена открытия (Open)
    * цена закрытия (ClosingPrice)
    * максимальная цена за день (DailyHigh)
    * минимальная цена за день (DailyLow) 
    * объем бумаг (VolumePcs)


In [1]:
#импортируем все библиотеки, которые нам потребуются
import re
import pandas as pd
import numpy as np
from dateutil.parser import parse
import pymorphy2

from nltk.corpus import stopwords
from pymystem3 import Mystem
from gensim.models import Phrases
from gensim import corpora, models, similarities

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB

from datetime import datetime
from sklearn.base import clone
from copy import copy

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

import nltk.corpus  
from nltk.text import Text

from scipy.stats.stats import pearsonr

Посмотрим на датасеты

In [2]:
df = pd.read_csv('texts.csv')
df.rename(columns={'date': 'Date', 'text': 'Text'}, inplace=True)
df.head()

,Date,Text
0,09.11.2017,Компания рассчитывает на решение по газовому с...
1,08.11.2017,"Как и предполагал “Ъ”, «Газпром», воспользова..."
2,01.11.2017,Новая редакция американских санкций ставит по...
3,30.10.2017,"Как стало известно “Ъ”, известный на рынке ри..."
4,23.10.2017,"НОВАТЭК, который через пять лет собирается за..."


In [3]:
pr_all = pd.read_csv('gazprom_prices.csv', sep=';')
pr_all.head()

,Date,Open,ClosingPrice,DailyHigh,DailyLow,VolumePcs
0,08.12.2017,"133,43000","132,60000","133,90000","132,00000",16037970
1,07.12.2017,"133,70000","133,02000","133,87000","132,81000",18198430
2,06.12.2017,"133,33000","134,00000","134,29000","132,91000",14641730
3,05.12.2017,"133,48000","133,65000","133,99000","132,78000",12684800
4,04.12.2017,"133,01000","133,77000","134,00000","131,93000",17818980


In [4]:
pr_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1988 entries, 0 to 1987
Data columns (total 6 columns):
Date            1988 non-null object
Open            1964 non-null object
ClosingPrice    1988 non-null object
DailyHigh       1986 non-null object
DailyLow        1986 non-null object
VolumePcs       1988 non-null int64
dtypes: int64(1), object(5)
memory usage: 93.3+ KB


Видим, что колонки, в которых по идее должны быть числа, на самом деле представлены другим типом данных. Запятая намекает на необходимость привести их к типу float, но перед этим саму запятую необходимо заменить на точку, иначе преобразование не получится.

In [5]:
for col in pr_all.columns[1:-1]:
    pr_all[col] = pr_all[col].apply(lambda x: float(str(x).replace(',', '.')))

pr_all.head()    

,Date,Open,ClosingPrice,DailyHigh,DailyLow,VolumePcs
0,08.12.2017,133.43,132.60,133.90,132.00,16037970
1,07.12.2017,133.70,133.02,133.87,132.81,18198430
2,06.12.2017,133.33,134.00,134.29,132.91,14641730
3,05.12.2017,133.48,133.65,133.99,132.78,12684800
4,04.12.2017,133.01,133.77,134.00,131.93,17818980


Чтобы было удобнее считать корреляцию, нам нужно объединить датасеты. В первую очередь мы должны привести дату в нужный формат. Начнем с датафрейма pr_all.

In [6]:
dates = [parse(date, dayfirst = True).date() for date in pr_all.Date]

In [7]:
pr_all.Date = dates
pr_all.head()

,Date,Open,ClosingPrice,DailyHigh,DailyLow,VolumePcs
0,2017-12-08,133.43,132.60,133.90,132.00,16037970
1,2017-12-07,133.70,133.02,133.87,132.81,18198430
2,2017-12-06,133.33,134.00,134.29,132.91,14641730
3,2017-12-05,133.48,133.65,133.99,132.78,12684800
4,2017-12-04,133.01,133.77,134.00,131.93,17818980


In [8]:
pr_all_date = pr_all.loc[:,['Date', 'ClosingPrice']]
pr_all_date.head()

,Date,ClosingPrice
0,2017-12-08,132.60
1,2017-12-07,133.02
2,2017-12-06,134.00
3,2017-12-05,133.65
4,2017-12-04,133.77


In [9]:
pr_all_date.shape

(1988, 2)

In [10]:
pr_all_date.ClosingPrice.isnull().sum()

0

Теперь нам нужно поработать с новостями.

In [11]:
df.head()

,Date,Text
0,09.11.2017,Компания рассчитывает на решение по газовому с...
1,08.11.2017,"Как и предполагал “Ъ”, «Газпром», воспользова..."
2,01.11.2017,Новая редакция американских санкций ставит по...
3,30.10.2017,"Как стало известно “Ъ”, известный на рынке ри..."
4,23.10.2017,"НОВАТЭК, который через пять лет собирается за..."


In [12]:
df.tail()

,Date,Text
1198,01.02.2010,"""Газпром"" не исключает в 2010 г. выпуска обли..."
1199,28.01.2010,"Консорциум во главе с российским ОАО ""Газпром..."
1200,19.01.2010,"""Газпром"" готов забирать весь объем азербайдж..."
1201,11.01.2010,Спорные вопросы по оплате за оказанные в пери...
1202,04.01.2010,


В конце есть дата без новости, нужно ее удалить.

In [13]:
df = df.iloc[:-1:]

In [14]:
df.tail()

,Date,Text
1197,03.02.2010,"Переговоры ОАО ""Газпром"" с оператором проекта..."
1198,01.02.2010,"""Газпром"" не исключает в 2010 г. выпуска обли..."
1199,28.01.2010,"Консорциум во главе с российским ОАО ""Газпром..."
1200,19.01.2010,"""Газпром"" готов забирать весь объем азербайдж..."
1201,11.01.2010,Спорные вопросы по оплате за оказанные в пери...


Теперь нам нужно для каждой новости опубликованной не в день торговли, присвоить следующий ближайший торговый день.

In [15]:
df_dates = [parse(date, dayfirst = True).date() for date in df.Date]
df_dates.reverse()

In [16]:
trade_dates = dates
trade_dates.reverse()

In [17]:
adj_news_dates = []

for news_date in df_dates:
    for trade_date in trade_dates:
        if news_date <= trade_date:
            adj_news_date = trade_date
            break
    adj_news_dates.append(adj_news_date)          

In [18]:
adj_news_dates.reverse()

In [19]:
df.Date = adj_news_dates
df.head()

,Date,Text
0,2017-11-09,Компания рассчитывает на решение по газовому с...
1,2017-11-08,"Как и предполагал “Ъ”, «Газпром», воспользова..."
2,2017-11-01,Новая редакция американских санкций ставит по...
3,2017-10-30,"Как стало известно “Ъ”, известный на рынке ри..."
4,2017-10-23,"НОВАТЭК, который через пять лет собирается за..."


Теперь нам нужно сопоставить новости с ценой закрытия.

In [20]:
news = df.sort_values('Date').set_index('Date')
stock = pr_all_date.sort_values('Date').set_index('Date')

Проверим на пропуски.

In [22]:
news.Text.isnull().sum()

0

In [23]:
stock.ClosingPrice.isnull().sum()

0

In [24]:
result = news.join(stock).sort_index()

In [25]:
result.head()

,Text,ClosingPrice
Date,,
2010-01-11,Спорные вопросы по оплате за оказанные в пери...,194.50
2010-01-19,"""Газпром"" готов забирать весь объем азербайдж...",189.76
2010-01-28,"Консорциум во главе с российским ОАО ""Газпром...",182.30
2010-02-01,"""Газпром"" не исключает в 2010 г. выпуска обли...",189.85
2010-02-03,"Переговоры ОАО ""Газпром"" с оператором проекта...",192.70


Делаем лемматизацию текстовых ячеек, но не убираем стоп слова пока, потому что для первой корреляции нам нужна переменная - количество слов в новостях.

In [26]:
result.Text = result.Text.apply(lambda x: str(x).lower())

In [27]:
result.head()

,Text,ClosingPrice
Date,,
2010-01-11,спорные вопросы по оплате за оказанные в пери...,194.50
2010-01-19,"""газпром"" готов забирать весь объем азербайдж...",189.76
2010-01-28,"консорциум во главе с российским оао ""газпром...",182.30
2010-02-01,"""газпром"" не исключает в 2010 г. выпуска обли...",189.85
2010-02-03,"переговоры оао ""газпром"" с оператором проекта...",192.70


В одном из семинаров были приведены функции, помогающие в предоработке текста

In [28]:
m = Mystem()
regex = re.compile("[А-Яа-яёA-Za-z]+")
mystopwords = stopwords.words('russian') + stopwords.words('english') + stopwords.words('german')
mystopwords += ['это', 'наш', 'т', 'д', 'этот', 'который', 'свой', 'наш', 'мой', 'твой', 'тот', 'делать', 'друг', 'сделать', 
                'любой', 'каждый', 'самый', 'хотеть', 'мочь', 'ваш', 'сей', 'весь', 'очень', 'день', 'год', 'м',
               'ч', 'руб', 'л', 'например', 'становиться', 'http', 'www', 'https', 'vk', 'vkontakte', 'vk', 'com', 'ru']  

In [29]:
def remove_url(text):
    text =  re.sub(r"http\S+", "", text).strip()
    text =  re.sub(r"vk\S+", "", text).strip()
    text =  re.sub(r"https\S+", "", text).strip()
    text =  re.sub(r"VK\S+", "", text).strip()
    return text

def words_only(text, regex=regex):
    return " ".join(regex.findall(text))

def remove_stopwords(text, mystopwords = mystopwords):
    try:
        return " ".join([token for token in text.split() if (token not in mystopwords) and (token.count(token[0]) != len(token))])
    except:
        return ""

In [30]:
def lemmatize(text, mystem=m):
    try:
        return "".join(m.lemmatize(text)).strip()  
    except:
        return ""

def length_filter(text, n_words = 3):
    tokens = text.split()
    if len(tokens) > n_words:
        return text
    else:
        return ""

def add_bigrams(texts, min_count = 5): # запись: "дональд_трамп"
    tokens = [text.split() for text in texts]
    bigram = Phrases(tokens, min_count = min_count)
    for i in range(len(tokens)):
        for token in bigram[tokens[i]]:
            if '_' in token:
                tokens[i].append(token)
    return [' '.join(text) for text in tokens]

Вся предобработка входных данных:

In [31]:
def preprocess(df):
    T = copy(df)
    T.Text = T.Text.str.lower()
    T.Text = T.Text.apply(remove_url)
    T.Text = T.Text.apply(words_only)
    T.Text.replace('', np.nan, inplace=True)
    T.dropna(subset=['Text'], inplace=True)
    T.Text = T.Text.apply(remove_stopwords)
    T.Text = T.Text.apply(lemmatize)
    T.Text = T.Text.apply(remove_stopwords)  
    T.Text = add_bigrams(T.Text)
    return T

На данном этапе мы не хотим удалять стоп-слова, поэтому преобразуем функцию preprocess()

In [32]:
def preprocess_with_stopwords(df):
    T = copy(df)
    T.Text = T.Text.str.lower()
    T.Text = T.Text.apply(remove_url)
    T.Text = T.Text.apply(words_only)
    T.Text.replace('', np.nan, inplace=True)
    T.dropna(subset=['Text'], inplace=True)
    T.Text = T.Text.apply(lemmatize)
    T.Text = add_bigrams(T.Text)
    return T

In [33]:
T = copy(result)

In [34]:
T_with_stopwords = preprocess_with_stopwords(T)
T_with_stopwords.head()

/Users/andrey_lukyanov/anaconda/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


,Text,ClosingPrice
Date,,
2010-01-11,спорный вопрос по оплата за оказывать в период...,194.50
2010-01-19,газпром готовый забирать весь объем азербайджа...,189.76
2010-01-28,консорциум во глава с российский оао газпром н...,182.30
2010-02-01,газпром не исключать в г выпуск облигация прим...,189.85
2010-02-03,переговоры оао газпром с оператор проект сахал...,192.70


Создаем новый столбец, который вычисляет длину новостного сообщения. Именно эта переменная нам и понадобится для первой корреляции. Но сначала нужно преобразовать тексты в список слов.

In [35]:
T_with_stopwords.Text = T_with_stopwords.Text.apply(lambda x: x.split())
T_with_stopwords.head()

,Text,ClosingPrice
Date,,
2010-01-11,"[спорный, вопрос, по, оплата, за, оказывать, в...",194.50
2010-01-19,"[газпром, готовый, забирать, весь, объем, азер...",189.76
2010-01-28,"[консорциум, во, глава, с, российский, оао, га...",182.30
2010-02-01,"[газпром, не, исключать, в, г, выпуск, облигац...",189.85
2010-02-03,"[переговоры, оао, газпром, с, оператор, проект...",192.70


In [36]:
T_with_stopwords['len_text'] = T_with_stopwords.Text.apply(lambda x: len(x))
T_with_stopwords.head()

,Text,ClosingPrice,len_text
Date,,,
2010-01-11,"[спорный, вопрос, по, оплата, за, оказывать, в...",194.50,66
2010-01-19,"[газпром, готовый, забирать, весь, объем, азер...",189.76,61
2010-01-28,"[консорциум, во, глава, с, российский, оао, га...",182.30,88
2010-02-01,"[газпром, не, исключать, в, г, выпуск, облигац...",189.85,39
2010-02-03,"[переговоры, оао, газпром, с, оператор, проект...",192.70,32


## Часть 1. Вводная [3 балла]

Проведите предобработку текстов: если считаете нужным, выполните токенизацию, приведение к нижнему регистру, лемматизацию и/или стемминг. Ответьте на следующие вопросы:
* Есть ли корреляция между средней длинной текста за день и ценой закрытия?
* Есть ли корреляция между количеством упоминаний Алексея Миллера  и ценой закрытия? Учтите разные варианты написания имени.
* Упоминаний какого газопровода в статьях больше: 
    * "северный поток"
    * "турецкий поток"?
* Кого упоминают чаще:
    * Алексея Миллера
    * Владимира Путина?
* О каких санкциях пишут в статьях?

# Корреляция

Приступаем непосредственно к ответу на вопросы.

### Есть ли корреляция между средней длинной текста за день и ценой закрытия?

Здесь сначала идет сам коэффициент, а потом его значимость. H0: Гипотеза об отсутствие какой-либо связи между двумя списками

In [53]:
x = list(T_with_stopwords['ClosingPrice'])
y = list(T_with_stopwords['len_text'])
corr = pearsonr(x,y)
print("Корреляция Пирсона:",corr[0])
print("p-value:", corr[1])

Корреляция Пирсона: 0.0136844534731
p-value: 0.635523570264


Вывод: У нас нет оснований отвергнуть гипотезу об отсутствии связи между ценой закрытия и средней длинной текста за день.Корреляция между ценой закрытия и средней длиной новостного сообщения отсутствует.

### Преобразование колонки с текстом в list, чтоб было удобней работать

In [54]:
text_list = T_with_stopwords['Text'].tolist()
print (text_list)

[['спорный', 'вопрос', 'по', 'оплата', 'за', 'оказывать', 'в', 'период', 'гг', 'услуга', 'по', 'транспортировка', 'природный', 'газ', 'по', 'территория', 'польша', 'должный', 'быть', 'урегулировать', 'на', 'основа', 'принцип', 'отсутствие', 'взаимный', 'претензия', 'переговоры', 'продолжаться', 'сообщать', 'риа', 'новость', 'в', 'управление', 'информация', 'оао', 'газпром', 'газпром', 'обещать', 'покупать', 'в', 'этот', 'год', 'весь', 'газ', 'который', 'захотеть', 'экспортировать', 'азербайджан', 'потенциальный', 'поставщик', 'сырье', 'для', 'конкурировать', 'с', 'российский', 'газопровод', 'южный', 'поток', 'проект', 'набукко', 'природный_газ', 'должный_быть', 'сообщать_риа', 'управление_информация', 'оао_газпром', 'газопровод_южный'], ['газпром', 'готовый', 'забирать', 'весь', 'объем', 'азербайджанский', 'газ', 'какой', 'баку', 'готовый', 'поставлять', 'заявлять', 'во', 'вторник', 'глава', 'газпром', 'алексей', 'миллер', 'специфика', 'контракт', 'с', 'азербайджан', 'заключаться', 'в'

In [55]:
def flatten(lst):
    result = []
    
    for outer_el in text_list:
        for inner_el in outer_el:
            result.append(inner_el)
  
    return result
  

text_list_1 = flatten(text_list)

Убираем стоп слова

In [56]:
from nltk.corpus import stopwords
nltk.download('stopwords')
mystopwords = stopwords.words('russian')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/andrey_lukyanov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Наиболее частотные биграммы и слова

In [57]:
filtered_words = [word for word in text_list_1 if word not in stopwords.words('russian')]
bg = list(nltk.bigrams(filtered_words))
bgfd = nltk.FreqDist(bg)
bgfd.most_common(50)

[(('газпром', 'нефть'), 675),
 (('млрд', 'руб'), 450),
 (('поставка', 'газ'), 189),
 (('газпром', 'медиа'), 181),
 (('оао', 'газпром'), 160),
 (('российский', 'газ'), 143),
 (('алексей', 'миллер'), 125),
 (('совет', 'директор'), 125),
 (('говориться', 'сообщение'), 123),
 (('млрд', 'кубометр'), 117),
 (('куб', 'м'), 109),
 (('цена', 'газ'), 100),
 (('чистый', 'прибыль'), 95),
 (('российский', 'компания'), 94),
 (('российский', 'газовый'), 91),
 (('нафтогаз', 'украина'), 91),
 (('председатель', 'правление'), 87),
 (('компания', 'газпром'), 82),
 (('природный', 'газ'), 80),
 (('составлять', 'млрд'), 80),
 (('nord', 'stream'), 80),
 (('млрд', 'куб'), 78),
 (('становиться', 'известно'), 74),
 (('газ', 'европа'), 72),
 (('газовый', 'холдинг'), 71),
 (('известно', 'ъ'), 70),
 (('как_становиться', 'известно_ъ'), 70),
 (('прошлый', 'год'), 69),
 (('газпром', 'энергохолдинг'), 69),
 (('млн', 'руб'), 69),
 (('данные', 'ъ'), 68),
 (('владимир', 'путин'), 67),
 (('газпром', 'мочь'), 65),
 (('холди

Преобразовываем в словарь список с биграммами и распределением их частот

In [58]:
bigramfreq = dict(bgfd)
len(bigramfreq)

62871

### Кого упоминают чаще: Алексея Миллера или Владимира Путина?

В этом задании можно попробовать 3 способа:
* искать имя и фамилию вместе
* искать только фамилию
* искать только имя

In [59]:
#биграммы
print("Алексей Миллер:", bigramfreq['алексей', 'миллер'], "Владимир Путин:", bigramfreq['владимир', 'путин'])

Алексей Миллер: 125 Владимир Путин: 67


In [60]:
bg_one_word = list(filtered_words)
word_freq = nltk.FreqDist(bg_one_word)
word_freq = dict(word_freq)

In [61]:
#фамилия
print("Миллер:", word_freq['миллер'], "Путин:", word_freq['путин'])

Миллер: 151 Путин: 73


In [62]:
#имя
print("Алексей:", word_freq['алексей'], "Владимир:", word_freq['владимир'])

Алексей: 156 Владимир: 97



Здесь сразу можем ответить на этот вопрос: Алексей Миллер упоминается чаще. Во всех статьях Алексей Миллер повторяется 125 раз, а Владимир Путин 67.

### Упоминаний какого газопровода в статьях больше: "северный поток" или "турецкий поток"?

In [63]:
print("Северный поток:", bigramfreq['северный', 'поток'], "Турецкий поток:", bigramfreq['турецкий', 'поток'])

Северный поток: 15 Турецкий поток: 39


Вывод: Турецкий поток упомянают чаще чем Северный поток

### Корреляция между количеством упоминаний Алексея Миллера и ценой закрытия

Добавим колонку с количеством упоминаний Алексея Миллера. Упоминанием будем считать наличие слова "Миллер".

In [64]:
T_with_stopwords['count_miller'] = T_with_stopwords.Text.apply(lambda x: x.count('миллер'))
T_with_stopwords.head()

,Text,ClosingPrice,len_text,count_miller
Date,,,,
2010-01-11,"[спорный, вопрос, по, оплата, за, оказывать, в...",194.50,66,0
2010-01-19,"[газпром, готовый, забирать, весь, объем, азер...",189.76,61,2
2010-01-28,"[консорциум, во, глава, с, российский, оао, га...",182.30,88,0
2010-02-01,"[газпром, не, исключать, в, г, выпуск, облигац...",189.85,39,0
2010-02-03,"[переговоры, оао, газпром, с, оператор, проект...",192.70,32,0


In [66]:
x = list(T_with_stopwords['ClosingPrice'])
y = list(T_with_stopwords['count_miller'])

corr = pearsonr(x,y)
print("Корреляция Пирсона:",corr[0])
print("p-value:", corr[1])

Корреляция Пирсона: 0.0107477220272
p-value: 0.709710290751


Вывод: Корреляция опять не значима. У нас нет оснований отвергнуть гипотезу об отсутствии связи между количеством упоминаний слова Миллер и ценой закрытия 

### О каких санкциях пишут в статьях?

In [67]:
text_list_2 = Text(text_list_1)
text_list_2.concordance('санкция', width=100)

Displaying 25 of 69 matches:
рона договариваться в г не применять штрафной санкция за нарушение контрактный обязательство по объ
зательство по транзит однако никакой штрафной санкция для монополия за это не предусматривать с вво
работать в страна подпадать под международный санкция на куба россия который не хотеть терять возмо
работать в страна подпадать под международный санкция на куба газпром_нефть разработка_месторождени
о чтобы устранять наш опасение компания ждать санкция пока еврокомиссия получать хороший комментари
 минэнерго уже несколько год пытаться вводить санкция к компания и они топ менеджер за неготовность
иллер мочь подпадать под визовый и финансовый санкция который европа планировать вводить в отношени
то касаться менеджер но после анализ ситуация санкция мочь распространяться и на деятельность росси
жание кредит из за вводить в отношение россия санкция сообщать президент газпром нефть александр дю
жание кредит из за вводить в отношение россия санкция сообщать президен

Во-первых, мы видим, что слово "санкция" упоминается всего 69 раз. Наиболее содержательными прилагательными к слову можно назвать "американский" и "западный". Еще часто встречается "санкция" "сша".

In [68]:
print("Американские санкции:", bigramfreq['американский', 'санкция'], "Западные санкции:",
      bigramfreq['западный', 'санкция'], "Санкции США:", bigramfreq['санкция', 'сша'])

Американские санкции: 1 Западные санкции: 7 Санкции США: 5


Мы видим, что примерно одинаковое количество упоминание западных санкций и санкций США.

В тексах в основном описывается каким образом западные(в лице ЕС) и американские санкции повлияли на работу Гаспрома, Гаспромбанка и других банков, как при этом изменились цены на нефть, каким образом Россия пережила три волны введения санкций(введение эмбарго). Выдим конкретный случай с санкциями против банка Владивосток-СПГ, Роснефти, в чем заключались эти санкции не совсем понятно из текста.  

## Часть 2. Классификационная [3 балла]
Вам предстоит решить следующую задачу: по текстам новостей за день определить, вырастет или понизится цена закрытия.
Для этого:
* бинаризуйте признак "цена закрытия":  новый признак ClosingPrice_bin равен 1, если по сравнению со вчера цена не упала, и 0 – в обратном случаея;
* составьте бучающее и тестовое множество: данные до начала 2016 года используются для обучения, данные с 2016 года и позже – для тестирования.

Таким образом, в каждлый момент времени мы знаем: 
* ClosingPrice_bin – бинарый целевой признак
* слова из статей, опубликованных в этот день – объясняющие признаки

В этой части задания вам нужно сделать baseline алгоритм и попытаться его улучшить в следующей части. 

Используйте любой известный вам алгоритм классификации текстов для того, Используйте $tf-idf$ преобразование, сингулярное разложение, нормировку признакого пространства и любые другие техники обработки данных, которые вы считаете нужным. Используйте accuracy и F-measure для оценки качества классификации. Покажите, как  $tf-idf$ преобразование или сингулярное разложение или любая другая использованная вами техника влияет на качество классификации.
Если у выбранного вами алгоритма есть гиперпараметры (например, $\alpha$ в преобразовании Лапласа для метода наивного Байеса), покажите, как изменение гиперпараметра влияет на качество классификации.

In [69]:
pr_all.head()

,Date,Open,ClosingPrice,DailyHigh,DailyLow,VolumePcs
0,2017-12-08,133.43,132.60,133.90,132.00,16037970
1,2017-12-07,133.70,133.02,133.87,132.81,18198430
2,2017-12-06,133.33,134.00,134.29,132.91,14641730
3,2017-12-05,133.48,133.65,133.99,132.78,12684800
4,2017-12-04,133.01,133.77,134.00,131.93,17818980


In [70]:
df.head()

,Date,Text
0,2017-11-09,Компания рассчитывает на решение по газовому с...
1,2017-11-08,"Как и предполагал “Ъ”, «Газпром», воспользова..."
2,2017-11-01,Новая редакция американских санкций ставит по...
3,2017-10-30,"Как стало известно “Ъ”, известный на рынке ри..."
4,2017-10-23,"НОВАТЭК, который через пять лет собирается за..."


Идея - создать колонку с lead-ом переменной (т.е. каждому значению переменной ClosingPrice будет соответвовать следующее значение этой же переменной). Это нужно для того, чтобы сравнить цену в разные дни. Но для того, чтобы мы могли вычитать, нужно предварительно привести значения этой переменной к типу float (сейчас тип string).

In [71]:
#смещаем значения на один день назад
pr_all['ClosingPrice_Delta'] = pr_all.shift(periods=-1, freq=None, axis=0)['ClosingPrice']

In [72]:
pr_all.head()

,Date,Open,ClosingPrice,DailyHigh,DailyLow,VolumePcs,ClosingPrice_Delta
0,2017-12-08,133.43,132.60,133.90,132.00,16037970,133.02
1,2017-12-07,133.70,133.02,133.87,132.81,18198430,134.00
2,2017-12-06,133.33,134.00,134.29,132.91,14641730,133.65
3,2017-12-05,133.48,133.65,133.99,132.78,12684800,133.77
4,2017-12-04,133.01,133.77,134.00,131.93,17818980,133.02


In [73]:
#Считаем дельту
pr_all['ClosingPrice_Delta'] = pr_all['ClosingPrice'] - pr_all['ClosingPrice_Delta']

In [74]:
pr_all.head()

,Date,Open,ClosingPrice,DailyHigh,DailyLow,VolumePcs,ClosingPrice_Delta
0,2017-12-08,133.43,132.60,133.90,132.00,16037970,-0.42
1,2017-12-07,133.70,133.02,133.87,132.81,18198430,-0.98
2,2017-12-06,133.33,134.00,134.29,132.91,14641730,0.35
3,2017-12-05,133.48,133.65,133.99,132.78,12684800,-0.12
4,2017-12-04,133.01,133.77,134.00,131.93,17818980,0.75


Для удобства четко пропишем условия формирования новой колонки:

* 1 - цена закрытия выросла или равна прошлому дню
* 0 - цена закрытия упала

In [75]:
#создаем новую колонку и заполняем ее нулями
pr_all['ClosingPrice_bin'] = 0

#заполняем бинарную переменную
mask = pr_all.ClosingPrice_Delta >= 0
column_name = 'ClosingPrice_bin'
pr_all.loc[mask, column_name] = 1

In [76]:
pr_all.head()

,Date,Open,ClosingPrice,DailyHigh,DailyLow,VolumePcs,ClosingPrice_Delta,ClosingPrice_bin
0,2017-12-08,133.43,132.60,133.90,132.00,16037970,-0.42,0
1,2017-12-07,133.70,133.02,133.87,132.81,18198430,-0.98,0
2,2017-12-06,133.33,134.00,134.29,132.91,14641730,0.35,1
3,2017-12-05,133.48,133.65,133.99,132.78,12684800,-0.12,0
4,2017-12-04,133.01,133.77,134.00,131.93,17818980,0.75,1


In [77]:
type(pr_all.Date[0])

datetime.date

Дату нам преобразовывать уже не нужно, так как мы сделали это в прошлом пункте.

In [78]:
stock_date = pr_all.loc[:,['Date', 'ClosingPrice_bin']]
stock_date.head()

,Date,ClosingPrice_bin
0,2017-12-08,0
1,2017-12-07,0
2,2017-12-06,1
3,2017-12-05,0
4,2017-12-04,1


In [79]:
stock_date.shape

(1988, 2)

In [80]:
stock_date.ClosingPrice_bin.isnull().sum()

0

In [81]:
df.head()

,Date,Text
0,2017-11-09,Компания рассчитывает на решение по газовому с...
1,2017-11-08,"Как и предполагал “Ъ”, «Газпром», воспользова..."
2,2017-11-01,Новая редакция американских санкций ставит по...
3,2017-10-30,"Как стало известно “Ъ”, известный на рынке ри..."
4,2017-10-23,"НОВАТЭК, который через пять лет собирается за..."


Теперь нам нужно сопоставить нововсти с бинарной переменной.

In [86]:
A = df.sort_values('Date').set_index('Date')
B = stock_date.sort_values('Date').set_index('Date')

In [87]:
result = A.join(B).sort_index()

In [88]:
result.ClosingPrice_bin.isnull().sum()

0

In [89]:
result.head()

,Text,ClosingPrice_bin
Date,,
2010-01-11,Спорные вопросы по оплате за оказанные в пери...,0
2010-01-19,"""Газпром"" готов забирать весь объем азербайдж...",0
2010-01-28,"Консорциум во главе с российским ОАО ""Газпром...",1
2010-02-01,"""Газпром"" не исключает в 2010 г. выпуска обли...",1
2010-02-03,"Переговоры ОАО ""Газпром"" с оператором проекта...",1


In [90]:
result['Date'] = result.index
result = result.set_index(np.arange(len(result)))
result.head()

,Text,ClosingPrice_bin,Date
0,Спорные вопросы по оплате за оказанные в пери...,0,2010-01-11
1,"""Газпром"" готов забирать весь объем азербайдж...",0,2010-01-19
2,"Консорциум во главе с российским ОАО ""Газпром...",1,2010-01-28
3,"""Газпром"" не исключает в 2010 г. выпуска обли...",1,2010-02-01
4,"Переговоры ОАО ""Газпром"" с оператором проекта...",1,2010-02-03


Отделим ClosingPrice_bin в переменную у. Тем самым получим вектор ответов

In [162]:
texts = result[['Text']]
texts.head()

,Text
0,Спорные вопросы по оплате за оказанные в пери...
1,"""Газпром"" готов забирать весь объем азербайдж..."
2,"Консорциум во главе с российским ОАО ""Газпром..."
3,"""Газпром"" не исключает в 2010 г. выпуска обли..."
4,"Переговоры ОАО ""Газпром"" с оператором проекта..."


In [163]:
y = result.ClosingPrice_bin
y.values

array([0, 0, 1, ..., 1, 1, 0])

In [164]:
texts.shape, y.shape

((1202, 1), (1202,))

### Нужно предоработать данные

Функции, помогающие преобразовать текст, были приведены в первом задании. Для удобства, приведем здесь итоговую. Вся предобработка входных данных:

In [165]:
def preprocess(df):
    T = copy(df)
    T.Text = T.Text.str.lower()
    T.Text = T.Text.apply(remove_url)
    T.Text = T.Text.apply(words_only)
    T.Text.replace('', np.nan, inplace=True)
    T.dropna(subset=['Text'], inplace=True)
    T.Text = T.Text.apply(remove_stopwords)
    T.Text = T.Text.apply(lemmatize)
    T.Text = T.Text.apply(remove_stopwords)  
    T.Text = add_bigrams(T.Text)
    return T

In [166]:
texts.head()

,Text
0,Спорные вопросы по оплате за оказанные в пери...
1,"""Газпром"" готов забирать весь объем азербайдж..."
2,"Консорциум во главе с российским ОАО ""Газпром..."
3,"""Газпром"" не исключает в 2010 г. выпуска обли..."
4,"Переговоры ОАО ""Газпром"" с оператором проекта..."


In [167]:
ptexts = preprocess(texts)
ptexts.head()

/Users/andrey_lukyanov/anaconda/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


,Text
0,спорный вопрос оплата оказывать период услуга ...
1,газпром готовый забирать объем азербайджанский...
2,консорциум глава российский оао газпром нефть ...
3,газпром исключать выпуск облигация примерно по...
4,переговоры оао газпром оператор проект сахалин...


In [338]:
ptexts.Text[0]

'спорный вопрос оплата оказывать период услуга транспортировка природный газ территория польша должный урегулировать основа принцип отсутствие взаимный претензия переговоры продолжаться сообщать риа новость управление информация оао газпром газпром обещать покупать газ захотеть экспортировать азербайджан потенциальный поставщик сырье конкурировать российский газопровод южный поток проект набукко природный_газ сообщать_риа управление_информация газопровод_южный'

## CountVectorizer

Для начала попробуем обучить модели по CountVectorizer. В качестве признаков оставим только слова, появляющиеся в текстах более 90 раз.

In [344]:
tf = CountVectorizer(analyzer='word', ngram_range=(1,1), min_df = 90, stop_words=mystopwords)
data = tf.fit_transform(ptexts.Text).toarray()
X = pd.DataFrame(data=data, columns=tf.get_feature_names())

Надо разбить тексты на обучающее и тестовое подмножества

In [345]:
result['Year'] = result.Date.apply(lambda x: x.year)

d = result.Date.values[0]

y2016 = np.where(result.Year == 2016)[0][0]
print('Индекс первого элемента, относящегося к 2016 году:', y2016)

X_train = X[:y2016]
X_test = X[y2016:]

y_train = y[:y2016]
y_test = y[y2016:]

Индекс первого элемента, относящегося к 2016 году: 935


In [346]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((935, 100), (267, 100), (935,), (267,))

Теперь обучим модель

In [314]:
def make_model(model, X_train, X_test, y_train, y_test):
    '''
    Обучает модель и оценивает ее качество (метрики: accuracy, f1-measure)
    
    Parameters
    ----------
    model - название модели (классификатора) с параметрами, например: model=MultinomialNB()
    X_train, y_train - обучающие подвыборки матрицы признаков Х и вектора ответов у
    X_test, y_test - тестовые подвыборки матрицы признаков Х и вектора ответов у
    
    '''
    
    startTime = datetime.now()
    temp_model = clone(model)
    y_pred = temp_model.fit(X_train, y_train).predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print('Accuracy:', accuracy)
    print('F1-measure:', f1)
    print('Затраченное время:', datetime.now() - startTime)
    print(y_pred)

In [350]:
#LogisticRegression
Cs = np.arange(1, 100, 10)

for C in Cs:
    lr = LogisticRegression(penalty="l2", fit_intercept=True, max_iter=100, C=C, solver="lbfgs", random_state=42)

    lr.fit(X_train, y_train)
    
    y_pred = lr.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print('C:', C, 'Accuracy:', accuracy, 'F1-measure:', f1) 

C: 1 Accuracy: 0.535580524345 F1-measure: 0.451327433628
C: 11 Accuracy: 0.516853932584 F1-measure: 0.431718061674
C: 21 Accuracy: 0.516853932584 F1-measure: 0.431718061674
C: 31 Accuracy: 0.516853932584 F1-measure: 0.431718061674
C: 41 Accuracy: 0.516853932584 F1-measure: 0.431718061674
C: 51 Accuracy: 0.516853932584 F1-measure: 0.431718061674
C: 61 Accuracy: 0.516853932584 F1-measure: 0.431718061674
C: 71 Accuracy: 0.516853932584 F1-measure: 0.431718061674
C: 81 Accuracy: 0.516853932584 F1-measure: 0.431718061674
C: 91 Accuracy: 0.516853932584 F1-measure: 0.431718061674


In [351]:
#MultinomialNB
alphas = np.arange(1, 100, 10)

for alpha in alphas:
    mnb = MultinomialNB(alpha=alpha)

    mnb.fit(X_train, y_train)
    
    y_pred = mnb.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print('Alpha:', alpha, 'Accuracy:', accuracy, 'F1-measure:', f1) 

Alpha: 1 Accuracy: 0.520599250936 F1-measure: 0.5
Alpha: 11 Accuracy: 0.52808988764 F1-measure: 0.5078125
Alpha: 21 Accuracy: 0.531835205993 F1-measure: 0.517374517375
Alpha: 31 Accuracy: 0.524344569288 F1-measure: 0.509652509653
Alpha: 41 Accuracy: 0.516853932584 F1-measure: 0.505747126437
Alpha: 51 Accuracy: 0.524344569288 F1-measure: 0.51711026616
Alpha: 61 Accuracy: 0.520599250936 F1-measure: 0.515151515152
Alpha: 71 Accuracy: 0.516853932584 F1-measure: 0.509505703422
Alpha: 81 Accuracy: 0.513108614232 F1-measure: 0.507575757576
Alpha: 91 Accuracy: 0.50936329588 F1-measure: 0.501901140684


In [353]:
#SGDClassifier
alphas = np.arange(0.1, 100, 10)

for alpha in alphas:
    sgdc = SGDClassifier(alpha=alpha)

    sgdc.fit(X_train, y_train)
    
    y_pred = sgdc.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print('Alpha:', alpha, 'Accuracy:', accuracy, 'F1-measure:', f1) 

Alpha: 0.1 Accuracy: 0.524344569288 F1-measure: 0.473029045643
Alpha: 10.1 Accuracy: 0.456928838951 F1-measure: 0.627249357326
Alpha: 20.1 Accuracy: 0.543071161049 F1-measure: 0.0
Alpha: 30.1 Accuracy: 0.456928838951 F1-measure: 0.627249357326
Alpha: 40.1 Accuracy: 0.456928838951 F1-measure: 0.627249357326
Alpha: 50.1 Accuracy: 0.456928838951 F1-measure: 0.627249357326
Alpha: 60.1 Accuracy: 0.543071161049 F1-measure: 0.0
Alpha: 70.1 Accuracy: 0.543071161049 F1-measure: 0.0
Alpha: 80.1 Accuracy: 0.543071161049 F1-measure: 0.0
Alpha: 90.1 Accuracy: 0.456928838951 F1-measure: 0.627249357326


/Users/andrey_lukyanov/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/andrey_lukyanov/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [318]:
from sklearn.svm import SVC

In [360]:
#SVC rbf
Cs = np.arange(0.1, 50, 5)

for C in Cs:
    svc = SVC(C=C, kernel='rbf', degree=3)

    svc.fit(X_train, y_train)
    
    y_pred = svc.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print('C:', C, 'Accuracy:', accuracy, 'F1-measure:', f1) 

/Users/andrey_lukyanov/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


C: 0.1 Accuracy: 0.543071161049 F1-measure: 0.0
C: 5.1 Accuracy: 0.513108614232 F1-measure: 0.507575757576
C: 10.1 Accuracy: 0.520599250936 F1-measure: 0.522388059701
C: 15.1 Accuracy: 0.513108614232 F1-measure: 0.511278195489
C: 20.1 Accuracy: 0.516853932584 F1-measure: 0.505747126437
C: 25.1 Accuracy: 0.505617977528 F1-measure: 0.492307692308
C: 30.1 Accuracy: 0.516853932584 F1-measure: 0.501930501931
C: 35.1 Accuracy: 0.50936329588 F1-measure: 0.490272373541
C: 40.1 Accuracy: 0.513108614232 F1-measure: 0.503816793893
C: 45.1 Accuracy: 0.505617977528 F1-measure: 0.5


In [361]:
#SVC poly
Cs = np.arange(0.1, 50, 5)

for C in Cs:
    svc = SVC(C=C, kernel='poly', degree=3)

    svc.fit(X_train, y_train)
    
    y_pred = svc.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print('C:', C, 'Accuracy:', accuracy, 'F1-measure:', f1) 

/Users/andrey_lukyanov/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


C: 0.1 Accuracy: 0.543071161049 F1-measure: 0.0
C: 5.1 Accuracy: 0.531835205993 F1-measure: 0.161073825503
C: 10.1 Accuracy: 0.539325842697 F1-measure: 0.185430463576
C: 15.1 Accuracy: 0.524344569288 F1-measure: 0.18064516129
C: 20.1 Accuracy: 0.516853932584 F1-measure: 0.218181818182
C: 25.1 Accuracy: 0.520599250936 F1-measure: 0.228915662651
C: 30.1 Accuracy: 0.516853932584 F1-measure: 0.254335260116
C: 35.1 Accuracy: 0.505617977528 F1-measure: 0.241379310345
C: 40.1 Accuracy: 0.501872659176 F1-measure: 0.248587570621
C: 45.1 Accuracy: 0.50936329588 F1-measure: 0.242774566474


In [367]:
from sklearn.ensemble import GradientBoostingClassifier

In [368]:
#GradientBoostingClassifier
n_estimators = np.array([10, 50, 100, 150, 200, 250, 300, 350])

for n_est in n_estimators:
    clf = GradientBoostingClassifier(n_estimators = n_est)

    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print('Estimators:', n_est, 'Accuracy:', accuracy, 'F1-measure:', f1) 

Estimators: 10 Accuracy: 0.550561797753 F1-measure: 0.259259259259
Estimators: 50 Accuracy: 0.505617977528 F1-measure: 0.492307692308
Estimators: 100 Accuracy: 0.50936329588 F1-measure: 0.490272373541
Estimators: 150 Accuracy: 0.498127340824 F1-measure: 0.472440944882
Estimators: 200 Accuracy: 0.50936329588 F1-measure: 0.478087649402
Estimators: 250 Accuracy: 0.505617977528 F1-measure: 0.488372093023
Estimators: 300 Accuracy: 0.516853932584 F1-measure: 0.494117647059
Estimators: 350 Accuracy: 0.539325842697 F1-measure: 0.502024291498


## TF-IDF

Пробуем то же самое с TF-IDF

In [369]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [370]:
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1,1), min_df = 90, stop_words=mystopwords)
data = tfidf.fit_transform(ptexts.Text).toarray()
X = pd.DataFrame(data=data, columns=tfidf.get_feature_names())

Надо разбить тексты на обучающее и тестовое подмножества

In [371]:
result['Year'] = result.Date.apply(lambda x: x.year)

d = result.Date.values[0]

y2016 = np.where(result.Year == 2016)[0][0]
print('Индекс первого элемента, относящегося к 2016 году:', y2016)

X_train = X[:y2016]
X_test = X[y2016:]

y_train = y[:y2016]
y_test = y[y2016:]

Индекс первого элемента, относящегося к 2016 году: 935


In [372]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((935, 100), (267, 100), (935,), (267,))

In [373]:
#LogisticRegression
Cs = np.arange(1, 100, 10)

for C in Cs:
    lr = LogisticRegression(penalty="l2", fit_intercept=True, max_iter=100, C=C, solver="lbfgs", random_state=42)

    lr.fit(X_train, y_train)
    
    y_pred = lr.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print('C:', C, 'Accuracy:', accuracy, 'F1-measure:', f1) 

C: 1 Accuracy: 0.513108614232 F1-measure: 0.475806451613
C: 11 Accuracy: 0.50936329588 F1-measure: 0.478087649402
C: 21 Accuracy: 0.505617977528 F1-measure: 0.472
C: 31 Accuracy: 0.516853932584 F1-measure: 0.490118577075
C: 41 Accuracy: 0.513108614232 F1-measure: 0.488188976378
C: 51 Accuracy: 0.513108614232 F1-measure: 0.488188976378
C: 61 Accuracy: 0.513108614232 F1-measure: 0.488188976378
C: 71 Accuracy: 0.513108614232 F1-measure: 0.488188976378
C: 81 Accuracy: 0.513108614232 F1-measure: 0.488188976378
C: 91 Accuracy: 0.513108614232 F1-measure: 0.488188976378


In [374]:
#MultinomialNB
alphas = np.arange(1, 100, 10)

for alpha in alphas:
    mnb = MultinomialNB(alpha=alpha)

    mnb.fit(X_train, y_train)
    
    y_pred = mnb.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print('Alpha:', alpha, 'Accuracy:', accuracy, 'F1-measure:', f1) 

Alpha: 1 Accuracy: 0.520599250936 F1-measure: 0.466666666667
Alpha: 11 Accuracy: 0.513108614232 F1-measure: 0.439655172414
Alpha: 21 Accuracy: 0.524344569288 F1-measure: 0.440528634361
Alpha: 31 Accuracy: 0.535580524345 F1-measure: 0.436363636364
Alpha: 41 Accuracy: 0.539325842697 F1-measure: 0.427906976744
Alpha: 51 Accuracy: 0.52808988764 F1-measure: 0.4
Alpha: 61 Accuracy: 0.52808988764 F1-measure: 0.376237623762
Alpha: 71 Accuracy: 0.520599250936 F1-measure: 0.353535353535
Alpha: 81 Accuracy: 0.516853932584 F1-measure: 0.345177664975
Alpha: 91 Accuracy: 0.513108614232 F1-measure: 0.329896907216


In [375]:
#SGDClassifier
alphas = np.arange(0.1, 100, 10)

for alpha in alphas:
    sgdc = SGDClassifier(alpha=alpha)

    sgdc.fit(X_train, y_train)
    
    y_pred = sgdc.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print('Alpha:', alpha, 'Accuracy:', accuracy, 'F1-measure:', f1) 

Alpha: 0.1 Accuracy: 0.430711610487 F1-measure: 0.577777777778
Alpha: 10.1 Accuracy: 0.456928838951 F1-measure: 0.627249357326
Alpha: 20.1 Accuracy: 0.456928838951 F1-measure: 0.627249357326
Alpha: 30.1 Accuracy: 0.456928838951 F1-measure: 0.627249357326
Alpha: 40.1 Accuracy: 0.456928838951 F1-measure: 0.627249357326
Alpha: 50.1 Accuracy: 0.543071161049 F1-measure: 0.0
Alpha: 60.1 Accuracy: 0.456928838951 F1-measure: 0.627249357326
Alpha: 70.1 Accuracy: 0.543071161049 F1-measure: 0.0
Alpha: 80.1 Accuracy: 0.456928838951 F1-measure: 0.627249357326
Alpha: 90.1 Accuracy: 0.543071161049 F1-measure: 0.0


/Users/andrey_lukyanov/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/andrey_lukyanov/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [376]:
from sklearn.svm import SVC

In [377]:
#SVC rbf
Cs = np.arange(0.1, 50, 5)

for C in Cs:
    svc = SVC(C=C, kernel='rbf', degree=3)

    svc.fit(X_train, y_train)
    
    y_pred = svc.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print('C:', C, 'Accuracy:', accuracy, 'F1-measure:', f1) 

/Users/andrey_lukyanov/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


C: 0.1 Accuracy: 0.543071161049 F1-measure: 0.0
C: 5.1 Accuracy: 0.543071161049 F1-measure: 0.031746031746
C: 10.1 Accuracy: 0.52808988764 F1-measure: 0.405660377358
C: 15.1 Accuracy: 0.558052434457 F1-measure: 0.495726495726
C: 20.1 Accuracy: 0.554307116105 F1-measure: 0.502092050209
C: 25.1 Accuracy: 0.531835205993 F1-measure: 0.476987447699
C: 30.1 Accuracy: 0.539325842697 F1-measure: 0.485355648536
C: 35.1 Accuracy: 0.535580524345 F1-measure: 0.487603305785
C: 40.1 Accuracy: 0.535580524345 F1-measure: 0.487603305785
C: 45.1 Accuracy: 0.531835205993 F1-measure: 0.48132780083


In [378]:
#SVC poly
Cs = np.arange(0.1, 50, 5)

for C in Cs:
    svc = SVC(C=C, kernel='poly', degree=3)

    svc.fit(X_train, y_train)
    
    y_pred = svc.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print('C:', C, 'Accuracy:', accuracy, 'F1-measure:', f1) 

/Users/andrey_lukyanov/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


C: 0.1 Accuracy: 0.543071161049 F1-measure: 0.0
C: 5.1 Accuracy: 0.543071161049 F1-measure: 0.0
C: 10.1 Accuracy: 0.543071161049 F1-measure: 0.0
C: 15.1 Accuracy: 0.543071161049 F1-measure: 0.0
C: 20.1 Accuracy: 0.543071161049 F1-measure: 0.0
C: 25.1 Accuracy: 0.543071161049 F1-measure: 0.0
C: 30.1 Accuracy: 0.543071161049 F1-measure: 0.0
C: 35.1 Accuracy: 0.543071161049 F1-measure: 0.0
C: 40.1 Accuracy: 0.543071161049 F1-measure: 0.0
C: 45.1 Accuracy: 0.543071161049 F1-measure: 0.0


In [379]:
from sklearn.ensemble import GradientBoostingClassifier

In [380]:
#GradientBoostingClassifier
n_estimators = np.array([10, 50, 100, 150, 200, 250, 300, 350])

for n_est in n_estimators:
    clf = GradientBoostingClassifier(n_estimators = n_est)

    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print('Estimators:', n_est, 'Accuracy:', accuracy, 'F1-measure:', f1) 

Estimators: 10 Accuracy: 0.505617977528 F1-measure: 0.4
Estimators: 50 Accuracy: 0.505617977528 F1-measure: 0.445378151261
Estimators: 100 Accuracy: 0.49063670412 F1-measure: 0.451612903226
Estimators: 150 Accuracy: 0.520599250936 F1-measure: 0.475409836066
Estimators: 200 Accuracy: 0.539325842697 F1-measure: 0.506024096386
Estimators: 250 Accuracy: 0.531835205993 F1-measure: 0.513618677043
Estimators: 300 Accuracy: 0.498127340824 F1-measure: 0.4765625
Estimators: 350 Accuracy: 0.501872659176 F1-measure: 0.478431372549


В среднем у CountVectorizer-выборки качество было немного выше. Все равно все модели плохие. Однако svc rbf по TF-IDF с параметром С = 20.1 показал точность 55.4 и F1 = 0.5. Будем считать, что это бейслайн.

## Часть 3. Творческая [4 балла]
Придумайте и попытайтесь сделать еще что-нибудь, чтобы улучшить качество классификации. 
Направления развития:
* Морфологический признаки: 
    * использовать в качестве признаков только существительные или только именованные сущности;
* Модели скрытых тем:
    * использовать в качестве признаков скрытые темы;
    * использовать в качестве признаков динамические скрытые темы 
    пример тут: (https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/dtm_example.ipynb)
* Синтаксические признаки:
    * использовать SOV-тройки в качестве признаков
    * кластеризовать SOV-тройки по усредненным эмбеддингам  (обученные word2vec модели можно скачать отсюда: (http://rusvectores.org/ru/models/ или https://github.com/facebookresearch/fastText/blob/master/pretrained-vectors.md) и использовать только центроиды кластеров в качестве признаков
* что-нибудь еще     

## Пробуем LSI

In [437]:
from gensim.models import LsiModel
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Doc2Vec

Будем сразу работать с данными CountVectorizer.

In [438]:
tf = CountVectorizer(analyzer='word', ngram_range=(1,1), min_df = 90, stop_words=mystopwords)
data = tf.fit_transform(ptexts.Text).toarray()
X = pd.DataFrame(data=data, columns=tf.get_feature_names())

Надо разбить тексты на обучающее и тестовое подмножества

In [439]:
ptexts['Year'] = result.Date.apply(lambda x: x.year)

d = result.Date.values[0]

y2016 = np.where(ptexts.Year == 2016)[0][0]
print('Индекс первого элемента, относящегося к 2016 году:', y2016)

X_train_init = ptexts[:y2016]
X_test_init = ptexts[y2016:]

y_train = y[:y2016]
y_test = y[y2016:]

Индекс первого элемента, относящегося к 2016 году: 935


In [440]:
X_train_init.shape, X_test_init.shape

((935, 2), (267, 2))

Некоторые фрагменты скрипта возьмем из одного из семинаров. Сначала преобразуем X_train.

In [441]:
texts = list(X_train_init.Text)

In [442]:
texts = [[word for word in text.split()] for text in texts]

from collections import defaultdict ## задаем частотный словарь
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1] for text in texts]

In [443]:
dictionary = corpora.Dictionary(texts) ## инициализируем словарь 
print(dictionary) 

Dictionary(4735 unique tokens: ['азербайджан', 'взаимный', 'вопрос', 'газ', 'газопровод']...)


In [444]:
corpus = [dictionary.doc2bow(text) for text in texts] ## здесь хранится непосрдественно векторная модель  

In [445]:
from gensim import corpora, models, similarities
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=10)
print(lsi)

LsiModel(num_terms=4735, num_topics=10, decay=1.0, chunksize=20000)


In [446]:
for i in lsi.print_topics():
    print(i)

(0, '0.705*"газпром" + 0.284*"компания" + 0.227*"газ" + 0.225*"нефть" + 0.195*"газпром_нефть" + 0.173*"млрд" + 0.140*"сообщать" + 0.115*"российский" + 0.089*"цена" + 0.085*"проект"')
(1, '0.549*"нефть" + 0.486*"газпром_нефть" + -0.433*"газ" + -0.143*"поставка" + 0.135*"компания" + -0.132*"украина" + -0.105*"российский" + -0.091*"газовый" + -0.087*"газопровод" + -0.073*"европа"')
(2, '0.681*"млрд" + 0.290*"прибыль" + 0.203*"чистый" + 0.179*"компания" + -0.119*"нефть" + 0.118*"оао" + -0.109*"газпром" + -0.108*"газ" + 0.106*"чистый_прибыль" + 0.102*"отчет"')
(3, '-0.449*"компания" + 0.408*"газ" + 0.294*"нефть" + 0.221*"цена" + 0.219*"газпром_нефть" + 0.207*"млрд" + 0.174*"поставка" + 0.146*"украина" + -0.129*"проект" + -0.120*"газпром"')
(4, '-0.519*"компания" + -0.384*"цена" + 0.244*"газпром" + 0.189*"млрд" + -0.168*"газ" + -0.131*"российский" + 0.126*"медиа" + 0.106*"сообщать" + -0.105*"монополия" + 0.103*"газпром_медиа"')
(5, '0.395*"проект" + -0.249*"сообщать" + 0.246*"роснефть" + -0.

In [447]:
vector = lsi[corpus]

corp_topics = []
for i in vector:
    doc = []
    for j in i:
        doc.append(j[1])
    corp_topics.append(doc)

In [448]:
X_train = np.array(corp_topics)

In [449]:
X_train.shape

(935, 10)

In [450]:
X_train.shape

(935, 10)

In [451]:
texts = list(X_test_init.Text)

In [452]:
texts = [[word for word in text.split()] for text in texts]

from collections import defaultdict ## задаем частотный словарь
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1] for text in texts]

In [453]:
dictionary = corpora.Dictionary(texts) ## инициализируем словарь 
print(dictionary) 

Dictionary(2119 unique tokens: ['актив', 'александр', 'газпром', 'газпром_медиа', 'гендиректор']...)


In [454]:
corpus = [dictionary.doc2bow(text) for text in texts] ## здесь хранится непосрдественно векторная модель  

In [455]:
vector = lsi[corpus]

corp_topics = []
for i in vector:
    doc = []
    for j in i:
        doc.append(j[1])
    corp_topics.append(doc)

In [456]:
X_test = np.array(corp_topics)

In [457]:
X_test.shape

(267, 10)

In [458]:
#SVC rbf
Cs = np.arange(0.1, 50, 5)

for C in Cs:
    svc = SVC(C=C, kernel='rbf', degree=3)

    svc.fit(X_train, y_train)
    
    y_pred = svc.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print('C:', C, 'Accuracy:', accuracy, 'F1-measure:', f1) 

/Users/andrey_lukyanov/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


C: 0.1 Accuracy: 0.543071161049 F1-measure: 0.0
C: 5.1 Accuracy: 0.449438202247 F1-measure: 0.599455040872
C: 10.1 Accuracy: 0.456928838951 F1-measure: 0.582132564841
C: 15.1 Accuracy: 0.47191011236 F1-measure: 0.595988538682
C: 20.1 Accuracy: 0.47191011236 F1-measure: 0.593659942363
C: 25.1 Accuracy: 0.445692883895 F1-measure: 0.55421686747
C: 30.1 Accuracy: 0.460674157303 F1-measure: 0.552795031056
C: 35.1 Accuracy: 0.460674157303 F1-measure: 0.547169811321
C: 40.1 Accuracy: 0.456928838951 F1-measure: 0.533762057878
C: 45.1 Accuracy: 0.464419475655 F1-measure: 0.531147540984


In [459]:
#GradientBoostingClassifier
n_estimators = np.array([10, 50, 100, 150, 200, 250, 300, 350])

for n_est in n_estimators:
    clf = GradientBoostingClassifier(n_estimators = n_est)

    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print('Estimators:', n_est, 'Accuracy:', accuracy, 'F1-measure:', f1) 

Estimators: 10 Accuracy: 0.498127340824 F1-measure: 0.556291390728
Estimators: 50 Accuracy: 0.50936329588 F1-measure: 0.55593220339
Estimators: 100 Accuracy: 0.50936329588 F1-measure: 0.516605166052
Estimators: 150 Accuracy: 0.505617977528 F1-measure: 0.541666666667
Estimators: 200 Accuracy: 0.468164794007 F1-measure: 0.510344827586
Estimators: 250 Accuracy: 0.494382022472 F1-measure: 0.529616724739
Estimators: 300 Accuracy: 0.505617977528 F1-measure: 0.535211267606
Estimators: 350 Accuracy: 0.516853932584 F1-measure: 0.544169611307


В результате наша точность стала ниже, но F1 мера в среднем повысилась.

## Попытка SVD

### Сначала работаем с CountVectorizer

In [460]:
tf = CountVectorizer(analyzer='word', ngram_range=(1,1), min_df = 90, stop_words=mystopwords)
data = tf.fit_transform(ptexts.Text).toarray()
X = pd.DataFrame(data=data, columns=tf.get_feature_names())

Надо разбить тексты на обучающее и тестовое подмножества

In [461]:
result['Year'] = result.Date.apply(lambda x: x.year)

d = result.Date.values[0]

y2016 = np.where(result.Year == 2016)[0][0]
print('Индекс первого элемента, относящегося к 2016 году:', y2016)

X_train = X[:y2016]
X_test = X[y2016:]

y_train = y[:y2016]
y_test = y[y2016:]

Индекс первого элемента, относящегося к 2016 году: 935


In [462]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((935, 100), (267, 100), (935,), (267,))

In [464]:
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline

In [497]:
components = np.arange(1, 50, 3)


for component in components:
    svd = TruncatedSVD(component)
    svc = SVC(C = 100, gamma = 10)
    
    x_train_svd = svd.fit_transform(X_train)
    x_test_svd = svd.fit_transform(X_test)

    svc.fit(x_train_svd, y_train)
    
    y_pred = svc.predict(x_test_svd)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print('Components:', component, 'Accuracy:', accuracy, 'F1-measure:', f1)    

Components: 1 Accuracy: 0.513108614232 F1-measure: 0.0972222222222
Components: 4 Accuracy: 0.505617977528 F1-measure: 0.56862745098
Components: 7 Accuracy: 0.513108614232 F1-measure: 0.475806451613
Components: 10 Accuracy: 0.524344569288 F1-measure: 0.485829959514
Components: 13 Accuracy: 0.483146067416 F1-measure: 0.420168067227
Components: 16 Accuracy: 0.468164794007 F1-measure: 0.366071428571
Components: 19 Accuracy: 0.49063670412 F1-measure: 0.392857142857
Components: 22 Accuracy: 0.468164794007 F1-measure: 0.418032786885
Components: 25 Accuracy: 0.520599250936 F1-measure: 0.448275862069
Components: 28 Accuracy: 0.513108614232 F1-measure: 0.356435643564
Components: 31 Accuracy: 0.550561797753 F1-measure: 0.387755102041
Components: 34 Accuracy: 0.543071161049 F1-measure: 0.364583333333
Components: 37 Accuracy: 0.520599250936 F1-measure: 0.311827956989
Components: 40 Accuracy: 0.524344569288 F1-measure: 0.211180124224
Components: 43 Accuracy: 0.539325842697 F1-measure: 0.263473053892

### Теперь пробуем с TF-IDF

In [467]:
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1,1), min_df = 90, stop_words=mystopwords)
data = tfidf.fit_transform(ptexts.Text).toarray()
X = pd.DataFrame(data=data, columns=tfidf.get_feature_names())

Надо разбить тексты на обучающее и тестовое подмножества

In [468]:
result['Year'] = result.Date.apply(lambda x: x.year)

d = result.Date.values[0]

y2016 = np.where(result.Year == 2016)[0][0]
print('Индекс первого элемента, относящегося к 2016 году:', y2016)

X_train = X[:y2016]
X_test = X[y2016:]

y_train = y[:y2016]
y_test = y[y2016:]

Индекс первого элемента, относящегося к 2016 году: 935


In [469]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((935, 100), (267, 100), (935,), (267,))

In [470]:
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline

In [499]:
components = np.arange(1, 50, 3)


for component in components:
    svd = TruncatedSVD(component)
    svc = SVC(C = 100, gamma = 10)
    
    x_train_svd = svd.fit_transform(X_train)
    x_test_svd = svd.fit_transform(X_test)

    svc.fit(x_train_svd, y_train)
    
    y_pred = svc.predict(x_test_svd)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print('Components:', component, 'Accuracy:', accuracy, 'F1-measure:', f1)    

Components: 1 Accuracy: 0.513108614232 F1-measure: 0.0972222222222
Components: 4 Accuracy: 0.498127340824 F1-measure: 0.570512820513
Components: 7 Accuracy: 0.513108614232 F1-measure: 0.471544715447
Components: 10 Accuracy: 0.539325842697 F1-measure: 0.489626556017
Components: 13 Accuracy: 0.494382022472 F1-measure: 0.415584415584
Components: 16 Accuracy: 0.456928838951 F1-measure: 0.366812227074
Components: 19 Accuracy: 0.49063670412 F1-measure: 0.37037037037
Components: 22 Accuracy: 0.516853932584 F1-measure: 0.41095890411
Components: 25 Accuracy: 0.531835205993 F1-measure: 0.444444444444
Components: 28 Accuracy: 0.501872659176 F1-measure: 0.398190045249
Components: 31 Accuracy: 0.558052434457 F1-measure: 0.397959183673
Components: 34 Accuracy: 0.573033707865 F1-measure: 0.43
Components: 37 Accuracy: 0.543071161049 F1-measure: 0.282352941176
Components: 40 Accuracy: 0.539325842697 F1-measure: 0.305084745763
Components: 43 Accuracy: 0.52808988764 F1-measure: 0.20253164557
Components: 

Все примерно одинаково плохо, но в случае данных CountVectorizer при 34 главных компонентах SVD показал точность 0.573033707865 и F-меру 0.43. В отношении точности это улучшило бейслайн.

## Сдача домашнего задания

Дедлайн сдачи домашнего задания:  23:59 22.03.2018. Каждый день просрочки дедлайна штрафуется -1 баллом.

Результаты домашнего задания должны быть оформлены в виде отчета в jupyter notebook.
Нормальный отчёт должен включать в себя:
* Краткую постановку задачи и формулировку задания
* Описание минимума необходимой теории и/или описание используемых инструментов 
* Подробный пошаговый рассказ о проделанной работе
* **Аккуратно** оформленные результаты
* Подробные и внятные ответы на все заданные вопросы 
* Внятные выводы – не стоит относится к домашнему заданию как к последовательности сугубо технических шагов, а стоит относится скорее как к небольшому практическому исследованию, у которого есть своя цель и свое назначение.

Задание выполняется в группе до трех человек. Не забудьте перечислить фамилии всех, кто работал над домашнем задании, в jupyter notebook.  

В случае использования какого-либо строннего источника информации обязательно дайте на него ссылку (поскольку другие тоже могут на него наткнуться). Плагиат наказывается нулём баллов за задание и предвзятым отношением в будущем.


При возникновении проблем с выполнением задания обращайтесь с вопросами к преподавателю по семинарским занятиям в вашей группе или у учебным ассистентам.

Учебный ассистент по ДЗ 2: Таисия Глушкова (email: glushkovato@gmail.com, telegram: @glushkovato).


Небрежное оформление отчета существенно отразится на итоговой оценке. Весь код из отчёта должен быть воспроизводимым, если для этого нужны какие-то дополнительные действия, установленные модули и т.п. — всё это должно быть прописано в отчете в явном виде.

Сдача отчетов осуществляется через систему AnyTask.

